In [1]:
import h5py    
import numpy as np    
import torch
from torch import nn
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
import random
# from tqdm.auto import tqdm
from tqdm.notebook import tqdm
from SVWN3 import f_svwn3
import csv
import copy

In [2]:
def ref(x, y):
    
    hartree2kcal = 627.5095
    with open("Reference_data.csv", newline='', encoding='cp1251') as csvfile:
        ref_file = csv.reader(csvfile, delimiter=",")
        k = 1
        if y == 391:
            k = hartree2kcal
        ref = []
        for n, i in enumerate(ref_file):
            if x <= n + 1 <= y:
                ref.append((i[0], float(i[2]) * k))

        return ref

def load_ref_energies():
    '''Returns {db_name: [equation, energy]}'''
    ref_e = { # Получение референсных энергий
        "MGAE109":ref(8, 116),
        "IP13":ref(155, 167),
        "EA13":ref(180, 192),
        "PA8":ref(195, 202),
        "DBH76":ref(251, 288) + ref(291, 328),
        "NCCE31":ref(331, 361),
        "ABDE4":ref(206, 209),
        "AE17":ref(375, 391),
        "pTC13":ref(232, 234) + ref(237, 241) + ref(244, 248)
        } 
    return ref_e

def load_component_names():
    '''Returns {db_name: {id: {
                                'Components': [...], 'Coefficients: [...]'
                                }
                            }
                        }
    '''
    with open("total_dataframe_sorted_final.csv", newline='', encoding='cp1251') as csvfile:
        ref_file = csv.reader(csvfile, delimiter=",")
        ref = dict()
        current_database = None
        
        for n, line in enumerate(ref_file):
            line = np.array(line)
            if n == 0:
                components = np.array(line)
            else:
                reaction_id = int(line[0])
                reaction_database = line[1]
                reaction_component_num = np.nonzero(list(map(float, line[2:])))[0] + 2
                if reaction_database in ref:
                    ref[reaction_database][reaction_id] = {'Components': components[reaction_component_num], 'Coefficients': line[reaction_component_num]}
                else: 
                    ref[reaction_database] = {reaction_id: {'Components': components[reaction_component_num], 'Coefficients': line[reaction_component_num]}}
        return ref

In [3]:
load_ref_energies()

{'MGAE109': [('CH(2ё)', 84.23),
  ('CH2(3B1)', 190.75),
  ('CH2(1A1)', 181.46),
  ("CH3(2A''2)", 307.88),
  ('CH4', 420.43),
  ('NH', 83.1),
  ('NH2', 182.59),
  ('NH3', 298.02),
  ('OH', 107.22),
  ('H2O', 232.98),
  ('HF', 141.63),
  ('SiH2(1A1)', 152.22),
  ('SiH2(3B1)', 131.48),
  ('SiH3', 228.01),
  ('SiH4', 324.95),
  ('PH2', 153.2),
  ('PH3', 242.27),
  ('H2S', 183.91),
  ('HCl', 107.5),
  ('C2H2', 405.53),
  ('CH2CH2', 563.69),
  ('CH3CH3', 712.98),
  ('CN', 181.36),
  ('HCN', 313.43),
  ('CO', 259.74),
  ('HCO', 279.43),
  ('H2CO', 374.67),
  ('CH3OH', 513.54),
  ('N2', 228.48),
  ('NH2NH2', 438.6),
  ('NO', 152.75),
  ('O2', 120.83),
  ('HOOH', 269.03),
  ('F2', 39.03),
  ('CO2', 390.16),
  ('Si2 (mult=3)', 76.38),
  ('P2', 117.59),
  ('S2', 104.25),
  ('Cl2', 59.75),
  ('SiO (mult=1)', 193.06),
  ('SC', 171.76),
  ('SO (m=3)', 126.48),
  ('ClO', 65.45),
  ('ClF', 62.79),
  ('Si2H6', 535.89),
  ('CH3Cl', 396.44),
  ('CH3SH', 474.49),
  ('HOCl', 166.24),
  ('SO2', 260.63),
  (

### Убеждаемся в том, что реакции и энергии идут в одном порядке на примере AE17


In [4]:
database_to_show = 'MGAE109'
print('Energies:')
load_ref_energies()[database_to_show]

Energies:


[('CH(2ё)', 84.23),
 ('CH2(3B1)', 190.75),
 ('CH2(1A1)', 181.46),
 ("CH3(2A''2)", 307.88),
 ('CH4', 420.43),
 ('NH', 83.1),
 ('NH2', 182.59),
 ('NH3', 298.02),
 ('OH', 107.22),
 ('H2O', 232.98),
 ('HF', 141.63),
 ('SiH2(1A1)', 152.22),
 ('SiH2(3B1)', 131.48),
 ('SiH3', 228.01),
 ('SiH4', 324.95),
 ('PH2', 153.2),
 ('PH3', 242.27),
 ('H2S', 183.91),
 ('HCl', 107.5),
 ('C2H2', 405.53),
 ('CH2CH2', 563.69),
 ('CH3CH3', 712.98),
 ('CN', 181.36),
 ('HCN', 313.43),
 ('CO', 259.74),
 ('HCO', 279.43),
 ('H2CO', 374.67),
 ('CH3OH', 513.54),
 ('N2', 228.48),
 ('NH2NH2', 438.6),
 ('NO', 152.75),
 ('O2', 120.83),
 ('HOOH', 269.03),
 ('F2', 39.03),
 ('CO2', 390.16),
 ('Si2 (mult=3)', 76.38),
 ('P2', 117.59),
 ('S2', 104.25),
 ('Cl2', 59.75),
 ('SiO (mult=1)', 193.06),
 ('SC', 171.76),
 ('SO (m=3)', 126.48),
 ('ClO', 65.45),
 ('ClF', 62.79),
 ('Si2H6', 535.89),
 ('CH3Cl', 396.44),
 ('CH3SH', 474.49),
 ('HOCl', 166.24),
 ('SO2', 260.63),
 ('AlCl3', 312.64),
 ('AlF3', 430.95),
 ('BCl3', 325.45),
 ('BF

In [5]:
print('Components:')
load_component_names()[database_to_show]

Components:


{0: {'Components': array(['C_mgae109', 'H_mgae109', 'CH_mgae109'], dtype='<U20'),
  'Coefficients': array(['1.0', '1.0', '-1.0'], dtype='<U7')},
 1: {'Components': array(['C_mgae109', 'H_mgae109', 'CH2_3B1_mgae109'], dtype='<U20'),
  'Coefficients': array(['1.0', '2.0', '-1.0'], dtype='<U7')},
 2: {'Components': array(['C_mgae109', 'H_mgae109', 'CH2_1A1_mgae109'], dtype='<U20'),
  'Coefficients': array(['1.0', '2.0', '-1.0'], dtype='<U7')},
 3: {'Components': array(['C_mgae109', 'H_mgae109', 'CH3_mgae109'], dtype='<U20'),
  'Coefficients': array(['1.0', '3.0', '-1.0'], dtype='<U7')},
 4: {'Components': array(['C_mgae109', 'H_mgae109', 'CH4_mgae109'], dtype='<U20'),
  'Coefficients': array(['1.0', '4.0', '-1.0'], dtype='<U7')},
 5: {'Components': array(['H_mgae109', 'N_mgae109', 'NH_mgae109'], dtype='<U20'),
  'Coefficients': array(['1.0', '1.0', '-1.0'], dtype='<U7')},
 6: {'Components': array(['H_mgae109', 'N_mgae109', 'NH2_mgae109'], dtype='<U20'),
  'Coefficients': array(['2.0', '1.

In [6]:
data = load_component_names()
databases = load_ref_energies().keys()
for database in databases:
    print(f'{database}: {len(load_ref_energies()[database])}, {len(load_component_names()[database])}')

MGAE109: 109, 109
IP13: 13, 13
EA13: 13, 13
PA8: 8, 8
DBH76: 76, 76
NCCE31: 31, 31
ABDE4: 4, 4
AE17: 17, 17
pTC13: 13, 13


In [ ]:
def get_compounds_coefs_energy_v2(reactions, energies):
    '''Returns {id: 
                    {'Components': [...], 'Coefficients: [...]', 'Energy: float', Database: str
                                }
                            }
    '''
    data_final = dict()
    i = 0
    databases = load_ref_energies().keys()
    for database in databases:
        data = reactions[database]
        for reaction in data:
            data[reaction]['Energy'] = energies[database][reaction][1]
            data_final[i] = {'Database': database,
                         'Components': reactions[database][reaction]['Components'],
                         'Coefficients': reactions[database][reaction]['Coefficients'],
                         'Energy': energies[database][reaction][1]
            
        }
            i += 1
        
    return data_final

data = get_compounds_coefs_energy_v2(load_component_names(), load_ref_energies())
data

{0: {'Database': 'MGAE109',
  'Components': array(['C_mgae109', 'H_mgae109', 'CH_mgae109'], dtype='<U20'),
  'Coefficients': array(['1.0', '1.0', '-1.0'], dtype='<U7'),
  'Energy': 84.23},
 1: {'Database': 'MGAE109',
  'Components': array(['C_mgae109', 'H_mgae109', 'CH2_3B1_mgae109'], dtype='<U20'),
  'Coefficients': array(['1.0', '2.0', '-1.0'], dtype='<U7'),
  'Energy': 190.75},
 2: {'Database': 'MGAE109',
  'Components': array(['C_mgae109', 'H_mgae109', 'CH2_1A1_mgae109'], dtype='<U20'),
  'Coefficients': array(['1.0', '2.0', '-1.0'], dtype='<U7'),
  'Energy': 181.46},
 3: {'Database': 'MGAE109',
  'Components': array(['C_mgae109', 'H_mgae109', 'CH3_mgae109'], dtype='<U20'),
  'Coefficients': array(['1.0', '3.0', '-1.0'], dtype='<U7'),
  'Energy': 307.88},
 4: {'Database': 'MGAE109',
  'Components': array(['C_mgae109', 'H_mgae109', 'CH4_mgae109'], dtype='<U20'),
  'Coefficients': array(['1.0', '4.0', '-1.0'], dtype='<U7'),
  'Energy': 420.43},
 5: {'Database': 'MGAE109',
  'Componen

In [ ]:
def drop_duplicates(data):
    pass

In [ ]:
def get_h5_names(reaction):
    '''reaction must be from the function get_compounds_coefs_energy_v2'''
    database_match = {
        'MGAE109': 'mgae109',
        'IP13': 'ip13',
        'EA13': 'ea13',
        'PA8': 'pa8',
        'DBH76': 'ntbh38',
        'NCCE31': 'ncce31',
        'ABDE4': 'abde4',
        'AE17': 'ae17',
        'pTC13': 'ptc13'
    }
    names = []
    for elem in reaction['Components']:
        database = database_match[reaction['Database']]
        names.append(f'{elem}.h5')
    return names

In [ ]:
reaction = data[73]
get_h5_names(reaction)

['H_mgae109.h5', 'H2_mgae109.h5']

In [ ]:
with h5py.File("H_mgae109.h5", "r") as f:
    y = np.array(f["ener"][:])
    X_raw = np.array(f["grid"][:])
X_raw[:,3] @ X_raw[:,-1]

-0.3088754803324676

In [ ]:
X_raw.shape

(33834, 12)

In [ ]:
X_raw[:,3]

array([2.57172640e-17, 9.97800968e-15, 3.26110049e-13, ...,
       2.60546656e-02, 2.26799037e-02, 2.60546656e-02])

In [ ]:
with h5py.File("H2_mgae109.h5", "r") as f:
    y = np.array(f["ener"][:])
    X_raw = np.array(f["grid"][:])
X_raw[:,3] @ X_raw[:,-1]

-0.6582181858640302

In [ ]:
X_raw.shape

(67668, 12)

In [ ]:
X_raw[:,3][33832]

0.002912519082071483

In [ ]:
X_raw[:,4:6]

array([[0.22047261, 0.22047261],
       [0.2204726 , 0.2204726 ],
       [0.22047252, 0.22047252],
       ...,
       [0.00033523, 0.00033523],
       [0.0004525 , 0.0004525 ],
       [0.00033523, 0.00033523]])

In [ ]:
def get_reaction_info_from_h5(reaction):
    '''
    reaction must be from get_compounds_coefs_energy_v2
    returns merged descriptos array X, integration weights, 
    a and b densities and indexes for backsplitting
    returns:
    X : np.array with grid descriptors
    weights : list with integration weights of grid points
    densities : np.array with alpha and beta densities data for grid points
    backsplit_ind: list of indexes where we concatenate molecules' grids
    '''
    X = np.array([]) 
    backsplit_ind = []
    HF_energies = np.array([])
    for component_filename in get_h5_names(reaction):
        with h5py.File(component_filename, "r") as f:
            np.append(HF_energies, f["ener"][:][0])
            X_raw = np.array(f["grid"][:])
            if len(X) == 0:
                X = X_raw[:, 3:-1]
            else:
                X = np.vstack((X, X_raw[:, 3:-1]))
            backsplit_ind.append(len(X))
    densities = X[:, 1:3]
    weights = X[:,0]
    X = X[:, 1:]
    return X, weights, densities, HF_energies, backsplit_ind

X, weights, densities, HF_energies, backsplit_ind = get_reaction_info_from_h5(reaction)

In [ ]:
reaction

{'Database': 'MGAE109',
 'Components': array(['H_mgae109', 'H2_mgae109'], dtype='<U20'),
 'Coefficients': array(['2.0', '-1.0'], dtype='<U7'),
 'Energy': 109.49}

In [252]:
densities

array([[0.31106103, 0.        ],
       [0.31106102, 0.        ],
       [0.3110609 , 0.        ],
       ...,
       [0.00033523, 0.00033523],
       [0.0004525 , 0.0004525 ],
       [0.00033523, 0.00033523]])

In [227]:
constants = [0.0310907, 0.01554535, 
                3.72744,   7.06042,
                12.9352,   18.0578,
                -0.10498,  -0.32500,
                0.0310907,  0.01554535,  -1/(6*np.pi**2),
                13.0720,    20.1231,      1.06835,
                42.7198,   101.578,      11.4813,
                -0.409286,  -0.743294,   -0.228344,
                1]
print(len(constants))
constants = np.tile(constants, [densities.shape[0],1])

21


In [228]:
def get_local_energies(constants, densities):
    energies = np.array([])
    
    for dens, consts in zip(densities, constants):
        energies = np.append(energies, f_svwn3(dens, consts))
    return energies

In [230]:
local_energies = get_local_energies(constants, densities)

In [233]:
def backsplit(reaction, X, backsplit_ind):

    splitted_data = dict()
    stop = 0
    for i, component in enumerate(reaction['Components']):
        start = stop
        stop = backsplit_ind[i]
        splitted_data[component] = X[start:stop]
    return splitted_data

In [234]:
splitted_energies = backsplit(reaction, local_energies, backsplit_ind)
splitted_energies

{'H_mgae109': array([-0.66306264, -0.66306264, -0.66306256, ..., -0.10956661,
        -0.11893653, -0.10956661]),
 'H2_mgae109': array([-0.62702259, -0.62702259, -0.62702251, ..., -0.0876153 ,
        -0.09582203, -0.0876153 ])}

In [294]:
splitted_energies['H_mgae109'].shape, splitted_energies['H2_mgae109'].shape

((33834,), (67668,))

In [296]:
splitted_weights = backsplit(reaction, weights, backsplit_ind)
splitted_weights

{'H_mgae109': array([2.57172640e-17, 9.97800968e-15, 3.26110049e-13, ...,
        2.60839681e-02, 2.27027326e-02, 2.60839681e-02]),
 'H2_mgae109': array([2.57172640e-17, 9.97800968e-15, 3.26110049e-13, ...,
        2.60546656e-02, 2.26799037e-02, 2.60546656e-02])}

In [299]:
splitted_densities = backsplit(reaction, densities, backsplit_ind)
splitted_densities

{'H_mgae109': array([[0.31106103, 0.        ],
        [0.31106102, 0.        ],
        [0.3110609 , 0.        ],
        ...,
        [0.00108441, 0.        ],
        [0.00140836, 0.        ],
        [0.00108441, 0.        ]]),
 'H2_mgae109': array([[0.22047261, 0.22047261],
        [0.2204726 , 0.2204726 ],
        [0.22047252, 0.22047252],
        ...,
        [0.00033523, 0.00033523],
        [0.0004525 , 0.0004525 ],
        [0.00033523, 0.00033523]])}

In [364]:
def integration(reaction, weights, energies, densities):
    molecule_energies = dict()
    for component in reaction['Components']:
        molecule_energies[component] = np.sum(energies[component] \
                                              * (densities[component][:,0]+densities[component][:,1]) \
                                              * (weights[component]))
    return molecule_energies

In [365]:
integration(reaction, splitted_weights, splitted_energies, splitted_densities)

{'H_mgae109': -0.28729038173416793, 'H2_mgae109': -0.6620570743354008}

In [383]:
for elem in [0, 1, 2, 3]:
    print(((elem % 3) + (elem%2) + (elem // 2) + (elem // 3) - (elem // 2)) / 2)

0.0
1.0
1.0
1.0


In [372]:
range(34)

range(0, 34)

In [ ]:
def train(model, criterion, optimizer, n_epochs=10):
    train_loss_mse = []
    train_loss_mae = []
    test_loss_mse = []
    test_loss_mae = []


    for epoch in range(n_epochs):
        # train
        model.train()
#         progress_bar = tqdm(train_loader)


        train_mse_losses_per_epoch = []
        train_mae_losses_per_epoch = []
        
        for X_batch, y_batch in progress_bar:


            X_batch, y_batch = X_batch.to(device), y_batch.to(device) # переехали на гпу
            predictions = model(X_batch) # смотрим че есть
            loss = criterion(predictions, y_batch) # оцениваем масштабы бедствия
            loss.backward() # обновляем градиенты
            optimizer.step() # делаем шаг градиентного спуска 
            optimizer.zero_grad()
            train_mse_losses_per_epoch.append(loss.item())
            train_mae_losses_per_epoch.append(mean_absolute_error(predictions.cpu().detach(), y_batch.cpu().detach()))
        train_loss_mse.append(np.mean(train_mse_losses_per_epoch))
        train_loss_mae.append(np.mean(train_mae_losses_per_epoch))


        #test
        model.eval()
        test_mse_losses_per_epoch = []
        test_mae_losses_per_epoch = []
        with torch.no_grad():

            for X_batch, y_batch in test_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                preds = model(X_batch)
                loss = criterion(preds, y_batch)
                test_mse_losses_per_epoch.append(loss.item())
                test_mae_losses_per_epoch.append(mean_absolute_error(preds.cpu().detach(), y_batch.cpu().detach()))
        test_loss_mse.append(np.mean(test_mse_losses_per_epoch))
        test_loss_mae.append(np.mean(test_mae_losses_per_epoch))
        print(f'train RMSE Loss = {train_loss_mse[epoch] ** 0.5:.4f}')
        print(f'train MAE Loss = {train_loss_mae[epoch]:.4f}')
        print(f'test RMSE Loss = {test_loss_mse[epoch] ** 0.5:.4f}')
        print(f'test MAE Loss = {test_loss_mae[epoch]:.4f}')
    return train_loss_mse, train_loss_mae, test_loss_mse, test_loss_mae, preds[0].cpu().detach().numpy()
        # print(np.array(train_accumulated_loss_mae).sum())
        # print('train RMSE Loss = {:.4f}'.format((train_accumulated_loss_mse. / len(X_train)) ** 0.5))
        # print('train MAE Loss = {:.4f}'.format((train_accumulated_loss_mae.sum() / len(X_train))))
        # print('test RMSE Loss = {:.4f}'.format((test_accumulated_loss_mse.sum() / len(X_test))) ** 0.5)

In [298]:
X.shape

(389592, 7)

In [288]:
with h5py.File('C2H2-C2H2_ncce31.h5', "r") as f:
    print(f["ener"])
    print(f['grid'])
    y = np.array(f["ener"][:])
    X = f['grid']

<HDF5 dataset "ener": shape (3,), type "<f8">
<HDF5 dataset "grid": shape (259728, 12), type "<f8">


In [292]:
with h5py.File('C6H6_mgae109.h5', "r") as f:
    print(f["ener"])
    print(f['grid'])
    y = np.array(f["ener"][:])
    X_raw = np.array(f["grid"][:])

<HDF5 dataset "ener": shape (3,), type "<f8">
<HDF5 dataset "grid": shape (389592, 12), type "<f8">


In [278]:
X = X_raw[:, 4:-1]

In [279]:
train_size = int(X.shape[0] * 0.8)
X_train = X[:train_size]
X_test = X[train_size:]


y_train_dist = [0.0310907, 0.01554535, 
                3.72744,   7.06042,
                12.9352,   18.0578,
                -0.10498,  -0.32500,
                0.0310907,  0.01554535,  -1/(6*np.pi**2),
                13.0720,    20.1231,      1.06835,
                42.7198,   101.578,      11.4813,
                -0.409286,  -0.743294,   -0.228344,
                1]

nconstants = len(y_train_dist)

# y_test_dist = np.random.normal(0, 1, nconstants)

In [280]:
y_train = np.tile(y_train_dist, [X_train.shape[0],1])
y_test = np.tile(y_train_dist, [X_test.shape[0],1])

In [281]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [282]:
# y_scaler = StandardScaler() 
# y_train = y_scaler.fit_transform(y_train[:, None])
# y_test = y_scaler.transform(y_test[:, None]);

In [283]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
set_random_seed(42)

In [284]:
device = torch.device('cuda:0') if torch.cuda.is_available else torch.device('cpu')
device

device(type='cuda', index=0)

In [285]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X).float()
        self.y = torch.tensor(y).float()
        
    def __getitem__(self, i):
        return self.X[i], self.y[i]
    
    def __len__(self):
        return len(self.X)

BATCH_SIZE = 1024

train_set = Dataset(X=X_train, y=y_train)
train_dataloader = torch.utils.data.DataLoader(train_set, 
                                            batch_size=BATCH_SIZE, shuffle=True, num_workers=4) # , num_workers=1

test_set = Dataset(X=X_test, y=y_test)
test_dataloader = torch.utils.data.DataLoader(test_set, 
                                            batch_size=BATCH_SIZE, num_workers=4) #, num_workers=1

In [15]:
class MLOptimizer(nn.Module):
    def __init__(self, nconstants):
        super().__init__()

        self.nconstants = nconstants
        self.hidden_layers = nn.Sequential(
                                nn.Linear(7, 256),
                                nn.BatchNorm1d(256),
                                nn.LeakyReLU(),
                                nn.Dropout(p=0.3),
                                nn.Linear(256, 256),
                                nn.BatchNorm1d(256),
                                nn.LeakyReLU(),
                                nn.Dropout(p=0.3),
                                nn.Linear(256, nconstants)
                            )
    def forward(self, input):
        x = self.hidden_layers(input)
        # x = f_svwn3(x)
        return x


In [16]:
model = MLOptimizer(nconstants=nconstants).to(device)

In [17]:
model.load_state_dict(torch.load('predoptimized.param'))

<All keys matched successfully>

In [18]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-6, betas=(0.9, 0.999))
criterion = nn.MSELoss()

In [22]:
def train(model, criterion, optimizer, train_loader, test_loader, n_epochs=10):
    train_loss_mse = []
    train_loss_mae = []
    test_loss_mse = []
    test_loss_mae = []


    for epoch in range(n_epochs):
        # train
        model.train()
        progress_bar = tqdm(train_loader)


        train_mse_losses_per_epoch = []
        train_mae_losses_per_epoch = []
        for X_batch, y_batch in progress_bar:


            X_batch, y_batch = X_batch.to(device), y_batch.to(device) # переехали на гпу
            predictions = model(X_batch) # смотрим че есть
            loss = criterion(predictions, y_batch) # оцениваем масштабы бедствия
            loss.backward() # обновляем градиенты
            optimizer.step() # делаем шаг градиентного спуска 
            optimizer.zero_grad()
            train_mse_losses_per_epoch.append(loss.item())
            train_mae_losses_per_epoch.append(mean_absolute_error(predictions.cpu().detach(), y_batch.cpu().detach()))
        train_loss_mse.append(np.mean(train_mse_losses_per_epoch))
        train_loss_mae.append(np.mean(train_mae_losses_per_epoch))


        #test
        model.eval()
        test_mse_losses_per_epoch = []
        test_mae_losses_per_epoch = []
        with torch.no_grad():

            for X_batch, y_batch in test_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                preds = model(X_batch)
                loss = criterion(preds, y_batch)
                test_mse_losses_per_epoch.append(loss.item())
                test_mae_losses_per_epoch.append(mean_absolute_error(preds.cpu().detach(), y_batch.cpu().detach()))
        test_loss_mse.append(np.mean(test_mse_losses_per_epoch))
        test_loss_mae.append(np.mean(test_mae_losses_per_epoch))
        print(f'train RMSE Loss = {train_loss_mse[epoch] ** 0.5:.4f}')
        print(f'train MAE Loss = {train_loss_mae[epoch]:.4f}')
        print(f'test RMSE Loss = {test_loss_mse[epoch] ** 0.5:.4f}')
        print(f'test MAE Loss = {test_loss_mae[epoch]:.4f}')
    return train_loss_mse, train_loss_mae, test_loss_mse, test_loss_mae, preds[0].cpu().detach().numpy()
        # print(np.array(train_accumulated_loss_mae).sum())
        # print('train RMSE Loss = {:.4f}'.format((train_accumulated_loss_mse. / len(X_train)) ** 0.5))
        # print('train MAE Loss = {:.4f}'.format((train_accumulated_loss_mae.sum() / len(X_train))))
        # print('test RMSE Loss = {:.4f}'.format((test_accumulated_loss_mse.sum() / len(X_test))) ** 0.5)

In [23]:
train_loss_mse, train_loss_mae, test_loss_mse, test_loss_mae, preds = train(model, criterion, optimizer, train_dataloader, test_dataloader)

  0%|          | 0/305 [00:00<?, ?it/s]

RuntimeError: DataLoader worker (pid(s) 27336, 21720, 19764, 13120) exited unexpectedly

In [ ]:
preds

In [ ]:
# print(train_loss_mse, train_loss_mae, test_loss_mse, test_loss_mae)
print('predicted coef', '\n', preds)
print('exact coef', '\n', np.array(y_train_dist))

In [20]:
torch.save(model.state_dict(), 'predoptimized.param')

In [24]:
def get_molecule_energy(molecule):
    energy = 0
    for point in molecule:
        energy += local_energy(constants)
    return energy

In [ ]:
def get_reaction_energy(product_enegergies, reagent_energies, product_coefficients, reagent_coefficients):
    return (product_enegergies @ product_coefficients) - (reagent_energies @ reagent_coefficients)

In [37]:
np.array([1, 2, 3]) @ np.array([3, 2, 1])

10

In [39]:
db = pd.read_csv('total_dataframe_sorted_final.csv')

In [40]:
db

ID in base Database  H_ae17  He_ae17  Li_ae17  Be_ae17  B_ae17  C_ae17  \
0             0     AE17     1.0      0.0      0.0      0.0     0.0     0.0   
1             1     AE17     0.0      1.0      0.0      0.0     0.0     0.0   
2             2     AE17     0.0      0.0      1.0      0.0     0.0     0.0   
3             3     AE17     0.0      0.0      0.0      1.0     0.0     0.0   
4             4     AE17     0.0      0.0      0.0      0.0     1.0     0.0   
..          ...      ...     ...      ...      ...      ...     ...     ...   
279          26   NCCE31     0.0      0.0      0.0      0.0     0.0     0.0   
280          27   NCCE31     0.0      0.0      0.0      0.0     0.0     0.0   
281          28   NCCE31     0.0      0.0      0.0      0.0     0.0     0.0   
282          29   NCCE31     0.0      0.0      0.0      0.0     0.0     0.0   
283          30   NCCE31     0.0      0.0      0.0      0.0     0.0     0.0   

     N_ae17  O_ae17  ...  CH4_ncce31  CH4-Ne_ncce31  C6H6_ncce31  \
0       0.0     0.0  ...         0.0            0.0          0.0   
1       0.0     0.0  ...         0.0            0.0          0.0   
2       0.0     0.0  ...         0.0            0.0          0.0   
3       0.0     0.0  ...         0.0            0.0          0.0   
4       0.0     0.0  ...         0.0            0.0          0.0   
..      ...     ...  ...         ...            ...          ...   
279     0.0     0.0  ...         0.0            0.0          0.0   
280     0.0     0.0  ...         0.0            0.0          0.0   
281     0.0     0.0  ...         0.0            0.0          2.0   
282     0.0     0.0  ...         0.0            0.0          2.0   
283     0.0     0.0  ...         0.0            0.0          2.0   

     C6H6-Ne_ncce31  CH4-CH4_ncce31  C2H2-C2H2_ncce31  C2H4-C2H4_ncce31  \
0               0.0             0.0               0.0               0.0   
1               0.0             0.0               0.0               0.0   
2               0.0             0.0               0.0               0.0   
3               0.0             0.0               0.0               0.0   
4               0.0             0.0               0.0               0.0   
..              ...             ...               ...               ...   
279             0.0             0.0              -1.0               0.0   
280             0.0             0.0               0.0              -1.0   
281             0.0             0.0               0.0               0.0   
282             0.0             0.0               0.0               0.0   
283             0.0             0.0               0.0               0.0   

     S-C6H6-C6H6_ncce31  T-C6H6-C6H6_ncce31  P-C6H6-C6H6_ncce31  
0                   0.0                 0.0                 0.0  
1                   0.0                 0.0                 0.0  
2                   0.0                 0.0                 0.0  
3                   0.0                 0.0                 0.0  
4                   0.0                 0.0                 0.0  
..                  ...                 ...                 ...  
279                 0.0                 0.0                 0.0  
280                 0.0                 0.0                 0.0  
281                -1.0                 0.0                 0.0  
282                 0.0                -1.0                 0.0  
283                 0.0                 0.0                -1.0  

[284 rows x 373 columns]

In [41]:
energies = pd.read_csv('Reference_data.csv')

In [48]:
energies

Reference data for the Minnesota Database (http://comp.chem.umn.edu/db)  \
0    REF1 is for direct comparison with calculated ...                        
1    REF2 is the "best estimate" for the considered...                        
2                                                  NaN                        
3                                                  NaN                        
4                                                CE345                        
..                                                 ...                        
535                                                NaN                        
536                                                NaN                        
537                                                NaN                        
538                                                NaN                        
539                                                NaN                        

    Unnamed: 1 Unnamed: 2                  Unnamed: 3  
0          NaN        NaN                         NaN  
1          NaN        NaN                         NaN  
2          NaN        NaN                         NaN  
3          NaN       REF1  REF2 (noVR): Best Estimate  
4          NaN        NaN                         NaN  
..         ...        ...                         ...  
535        NaN        NaN                         NaN  
536        NaN        NaN                         NaN  
537        NaN        NaN                         NaN  
538        NaN        NaN                         NaN  
539        NaN        NaN                         NaN  

[540 rows x 4 columns]

In [49]:
energies.dropna(how='all')

Reference data for the Minnesota Database (http://comp.chem.umn.edu/db)  \
0    REF1 is for direct comparison with calculated ...                        
1    REF2 is the "best estimate" for the considered...                        
3                                                  NaN                        
4                                                CE345                        
5                                              MGAE109                        
..                                                 ...                        
513                                               MgSe                        
514                                               MgTe                        
515                                                BaS                        
516                                               BaSe                        
517                                               BaTe                        

    Unnamed: 1 Unnamed: 2                  Unnamed: 3  
0          NaN        NaN                         NaN  
1          NaN        NaN                         NaN  
3          NaN       REF1  REF2 (noVR): Best Estimate  
4          NaN        NaN                         NaN  
5          NaN        NaN                         NaN  
..         ...        ...                         ...  
513         B1      5.375                         NaN  
514         B3      6.410                         NaN  
515         B1      6.364                         NaN  
516         B1      6.570                         NaN  
517         B1      6.982                         NaN  

[488 rows x 4 columns]

In [6]:
from torch import nn
import torch

nn.ReLU()()

ReLU(inplace=True)